# 完成版

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# Amazon商品ページを開く
# 固定入力パターン
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083271&dc&fs=true&ds=v1%3AjNxS2PZfTpTxxP1weAHzLyoVu6VNah9Uop7hosBt8fU&qid=1686742341&rnid=3895771&ref=sr_nr_n_1'
# 手動入力パターン
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
# print('')
browser.get(url)
browser.implicitly_wait(5)

# Amazon のウィンドウハンドラの保持
amazon_window = browser.current_window_handle

# 開いた Amazon ページからヤフオク商品ページを開く関数
def open_yafuoku_page():
    # Amazon商品＋ヤフオク商品群の基点要素を取得する
    amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    # Amazon＋ヤフオク商品群の親要素内の情報を取得する
    for amazon_elem in amazon_yafuoku_item_list:

        # Amazon商品名
        amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
        # print('Amazon商品名：', amazon_name)

        # Amazon価格
        try:
            amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
            # print('Amazon価格：', amazon_price, '円')
        except:
            amazon_price = 'Amazon価格情報なし'
            # print('▲ Amazon商品価格なし ▲')

        # ヤフオク商品群の基点要素を取得する
        yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')

        # ヤフオク商品群の親要素内の情報を取得する
        for yafuoku_elem in yafuoku_item_list:

            print('Amazon商品名：', amazon_name)
            print('Amazon価格：', amazon_price)

            # ヤフオク商品名
            try:
                yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                print('ヤフオク商品名：', yafuoku_name)
            except:
                print('▲ 該当ヤフオク商品なし ▲')
                # pass

            # ヤフオク現在価格
            try:
                yafuoku_price_now = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                print('ヤフオク現在価格：', yafuoku_price_now)
            except:
                print('▲ ヤフオク現在価格なし ▲')
                # pass

            # 現在価格での利益額計算
            profit_now = amazon_price - yafuoku_price_now
            print('A-Y現在価格差(送料考慮前)：', profit_now)

            # 原罪価格での利益率計算
            profit_rate_now = profit_now / yafuoku_price_now
            print('A-Y現在利益率(送料考慮前)：', round(profit_rate_now*100, 1), '%')

            # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
            if profit_rate_now > 0.1: # 利益率での判定

                # ヤフオク商品ページを開く
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)

                print('---------------------------------------------------------------------------------')
                print('開いたヤフオク商品への対応を選んでください( YES / NO / END)')
                print('　YES：商品情報を取得する')
                print('　NO ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)')
                print('　END：プログラムを終了する')
                print('---------------------------------------------------------------------------------')
                
                while True:
                    ans = input()

                    if ans == 'YES':
                        # print('この商品の情報を取得します')
                        # ヤフオク商品ページの情報取得に移る
                        get_yafuoku_data()
                        break
                    elif ans == 'NO':
                        print('この商品の情報取得はスキップしました')
                        # 開いたヤフオク商品ページをアクティブにしてから閉じて Amazon ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        break
                    elif ans == 'END':
                        print('プログラムを終了します')
                        # ヤフオク商品ページと Amazon ページを閉じてプログラムを終了する
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        browser.close()
                        sys.exit()
                    else:
                        print('YES か NO か END を入力してください')
            else:
                print('◆ 利益率が基準値未満のため情報取得をスキップしました ◆')
            
            print('')
        print('')
        
# ヤフオク商品ページから各種情報を取得する関数
def get_yafuoku_data():
    # 新たに開いたヤフオク商品ページをアクティブにする
    handle_array = browser.window_handles
    browser.switch_to.window(handle_array[-1])

    yafuoku_window = browser.current_window_handle
    # print('最新のヤフオク商品ページをアクティブにしました')

    # ヤフオク商品名を取得する
    yafuoku_name = browser.find_element(By.XPATH, '//h1').text
    print('ヤフオク商品名：', yafuoku_name)

    # 出品者の評価
    # 97％以下の場合は処理をスキップする
    seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
    print('評価：', seller_rating)

    seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
    
    if seller_rating_int < 97:
        print('◆ 評価が低いため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return

    # 評価数は多い方が望ましい
    seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
    print('評価数：', seller_rating_num)

    # 匿名配送情報
    # 匿名配送希望の場合は処理をスキップする
    try:
        # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
        privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
        print('◆ 匿名配送のため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return
    except:
        privacy = '匿名配送なし'
        print('匿名配送：', privacy)

    # ヤフオク商品ページ（アクティブウィンドウ）の URL を取得する
    yafuoku_url = browser.current_url
    print('ヤフオク商品URL：', yafuoku_url)
        
    # ヤフオク現在価格を取得する
    yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
    print('ヤフオク現在価格：', yafuoku_price_now)

    # 後の利益計算のために int 型にしておく
    if '税込' in yafuoku_price_now:
        yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
    else:
        yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])
        
    # 商品の状態
    state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
    print('状態：', state)
    
    # 入札数
    bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
    print('入札数：', bidnum)

    browser.close()
    browser.switch_to.window(amazon_window)

    # 送料
    # ★
    
    # 商品説明
    # ★
    
    # 直接引き取り判定
    # ★

    # 着払い判定
    # ★

    # 配送業者、営業所止め判定
    # ★

    # 訳あり品判定
    # ★

    # オークション残り日数
    # ★

    # ヤフオク送料も加味した、現在価格での利益額/利益率判定　※ Amazon 出品送料は ResaleTrap で判定
    # ★

    # 取得したデータをリストに格納する
    # ■ どうやる？？？
    # store_data()
    # return amazon_name

# 取得した情報を CSV 出力用にまとめる（出品判定の証跡として残す）関数
def store_data():
    get_datetime = datetime.datetime.now().strftime('%Y/%m/%d %H:%M')

    data = {
        'Amazon商品名': amazon_name,
        # 'Amazon価格': amazon_price,
        # 'Amazon商品URL': amazon_url,
        # 'ヤフオク商品名': yafuoku_name,
        # 'ヤフオク現在価格': yafuoku_price_now,
        # 'ヤフオク商品URL': yafuoku_url,
        # 'A-Y現在利益(送料込)': profit,
        # '現在利益率(送料込)': profit_rate,
        # '情報取得日時': get_datetime,
        }

    data_list.append(data)
    print('data_list', data_list)

# 取得した情報を CSV ファイルに出力する関数
def output():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
        
def main():
    open_yafuoku_page()
    # store_data()
    # output()

if __name__ == '__main__':
    main()

C:\Users\amuza\AppData\Local\Temp\ipykernel_8980\432575864.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： (中古品)iwaki(イワキ) ボウル 耐熱ガラス 丸型 1.5L 外径21.3cm KBC323
ヤフオク現在価格： 3480
A-Y現在価格差(送料考慮前)： 26420
A-Y現在利益率(送料考慮前)： 759.2 %
---------------------------------------------------------------------------------
開いたヤフオク商品への対応を選んでください( YES / NO / END)
　YES：商品情報を取得する
　NO ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)
　END：プログラムを終了する
---------------------------------------------------------------------------------


 YES


ヤフオク商品名： (中古品)iwaki(イワキ) ボウル 耐熱ガラス 丸型 1.5L 外径21.3cm KBC323
評価： 75.0%
◆ 評価が低いため本商品の情報収集を終了します ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： オーブン レンジ
ヤフオク現在価格： 6526
A-Y現在価格差(送料考慮前)： 23374
A-Y現在利益率(送料考慮前)： 358.2 %
---------------------------------------------------------------------------------
開いたヤフオク商品への対応を選んでください( YES / NO / END)
　YES：商品情報を取得する
　NO ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)
　END：プログラムを終了する
---------------------------------------------------------------------------------


 YES


ヤフオク商品名： オーブン レンジ
評価： 98.6%
評価数： 215
匿名配送： 匿名配送なし
ヤフオク商品URL： https://page.auctions.yahoo.co.jp/jp/auction/v711148276
ヤフオク現在価格： 6526円（税0円）
状態： 傷や汚れあり
入札数： 0件

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： パナソニック オーブンレンジ 26L フラットテーブル 遠赤Wヒーター スイン
ヤフオク現在価格： 28644
A-Y現在価格差(送料考慮前)： 1256
A-Y現在利益率(送料考慮前)： 4.4 %
◆ 利益率が基準値未満のため情報取得をスキップしました ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： パナソニック オーブンレンジ 26L フラットテーブル 遠赤Wヒーター スイン
ヤフオク現在価格： 34472
A-Y現在価格差(送料考慮前)： -4572
A-Y現在利益率(送料考慮前)： -13.3 %
◆ 利益率が基準値未満のため情報取得をスキップしました ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： IWAKI耐熱ガラスボウル②
ヤフオク現在価格： 500
A-Y現在価格差(送料考慮前)： 29400
A-Y現在利益率(送料考慮前)： 5880.0 %
---------------------------------------------------------------------------------
開いたヤフオク商品へ

 NO


この商品の情報取得はスキップしました

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： パナソニック オーブンレンジ 26L フラットテーブル 遠赤Wヒーター スイングサーチ赤外線センサー ブラック NE-MS268-K
ヤフオク現在価格： 40145
A-Y現在価格差(送料考慮前)： -10245
A-Y現在利益率(送料考慮前)： -25.5 %
◆ 利益率が基準値未満のため情報取得をスキップしました ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： パナソニック エレック オーブンレンジ 26L フラットテーブル 遠赤Wヒーター スイングサーチ赤外線センサー ブラック (中古品)
ヤフオク現在価格： 32469
A-Y現在価格差(送料考慮前)： -2569
A-Y現在利益率(送料考慮前)： -7.9 %
◆ 利益率が基準値未満のため情報取得をスキップしました ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： パナソニック エレック オーブンレンジ 26L フラットテーブル 遠赤Wヒーター スイングサーチ赤外線センサー ブラック NE-MS266-
ヤフオク現在価格： 29224
A-Y現在価格差(送料考慮前)： 676
A-Y現在利益率(送料考慮前)： 2.3 %
◆ 利益率が基準値未満のため情報取得をスキップしました ◆

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型